# Data Exploration - CLOB Risk Modelling

This notebook explores market data from Binance (CLOB) and Uniswap (AMM) to understand liquidity profiles and volatility characteristics.

In [ ]:
# Setup and imports
import sys
import os
sys.path.append(os.path.abspath('../src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from risk_model import config, binance_data, uniswap_data, pricing, plotting

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Modules loaded successfully!")

## 1. Load Configuration

In [ ]:
# Load markets configuration
markets_config = config.load_markets()
risk_params = config.get_risk_parameters()

print(f"Loaded {len(markets_config['markets'])} markets:")
for market in markets_config['markets']:
    print(f"  - {market['name']}: {market['binance_symbol_spot']} (Category: {market['category']})")

print(f"\nRisk parameters loaded:")
print(f"  - Max price impact: {risk_params['max_price_impact_pct']}%")
print(f"  - OI/Volume warning ratio: {risk_params['oi_to_volume_warning_ratio']}x")

## 2. Fetch Binance Data

In [ ]:
# Select market to analyze
market = markets_config['markets'][0]  # ETH-PERP
symbol = market['binance_symbol_perp']

print(f"Fetching data for {market['name']} ({symbol})...")

# Initialize fetcher
binance = binance_data.BinanceDataFetcher()

# Fetch orderbook
orderbook = binance.get_orderbook(symbol, limit=500, use_futures=True)
print(f"\nOrderbook snapshot:")
print(f"  - Bids: {len(orderbook['bids'])} levels")
print(f"  - Asks: {len(orderbook['asks'])} levels")
print(f"  - Best bid: ${orderbook['bids'][0][0]:.2f}")
print(f"  - Best ask: ${orderbook['asks'][0][0]:.2f}")
print(f"  - Spread: ${orderbook['asks'][0][0] - orderbook['bids'][0][0]:.4f}")

In [ ]:
# Fetch historical data
klines_1h = binance.get_klines(symbol, interval='1h', lookback_days=30, use_futures=True)
klines_1d = binance.get_klines(symbol, interval='1d', lookback_days=90, use_futures=True)

print(f"Historical data:")
print(f"  - Hourly: {len(klines_1h)} candles ({klines_1h.index[0]} to {klines_1h.index[-1]})")
print(f"  - Daily: {len(klines_1d)} candles ({klines_1d.index[0]} to {klines_1d.index[-1]})")
print(f"\nCurrent price: ${klines_1h['close'].iloc[-1]:.2f}")

In [ ]:
# Fetch 24h ticker data
ticker_24h = binance.get_ticker_24h(symbol, use_futures=True)
book_ticker = binance.get_book_ticker(symbol, use_futures=True)

print(f"24h Statistics:")
print(f"  - Volume: ${float(ticker_24h['quoteVolume']):,.0f}")
print(f"  - High: ${float(ticker_24h['highPrice']):.2f}")
print(f"  - Low: ${float(ticker_24h['lowPrice']):.2f}")
print(f"  - Price change: {float(ticker_24h['priceChangePercent']):.2f}%")

spread_metrics = pricing.calculate_spread_metrics(book_ticker)
print(f"\nSpread metrics:")
print(f"  - Mid price: ${spread_metrics['mid_price']:.2f}")
print(f"  - Spread (bps): {spread_metrics['spread_bps']:.1f}")

## 3. Analyze Orderbook Liquidity

In [ ]:
# Compute liquidity curves
liquidity_curve = pricing.compute_orderbook_liquidity_curve(orderbook, max_impact_pct=5.0)

# Calculate depth at different impact levels
impact_levels = [0.1, 0.5, 1.0, 2.0, 5.0]
depths = {}

for impact in impact_levels:
    depth = pricing.depth_at_impact(liquidity_curve, impact, side='both')
    depths[impact] = depth
    print(f"Depth at {impact}% impact: ${depth:,.0f}")

In [ ]:
# Visualize orderbook depth
fig, axes = plotting.plot_orderbook_depth(liquidity_curve, title=f"{symbol} Orderbook")
plt.show()

## 4. Fetch Uniswap Data

In [ ]:
# Fetch Uniswap pool data
pool_id = market['uniswap_pool_id']
print(f"Fetching Uniswap v3 pool data for {pool_id}...")

try:
    uniswap = uniswap_data.UniswapDataFetcher()
    pool_data = uniswap.query_pool(pool_id)
    
    if pool_data:
        print(f"\nPool information:")
        print(f"  - Pair: {pool_data['token0']['symbol']}/{pool_data['token1']['symbol']}")
        print(f"  - Fee tier: {int(pool_data['feeTier'])/10000}%")
        print(f"  - TVL: ${float(pool_data['totalValueLockedUSD']):,.0f}")
        print(f"  - 24h volume: ${float(pool_data.get('volumeUSD', 0)):,.0f}")
        print(f"  - Current price: ${float(pool_data['token0Price']):.2f}")
        
        # Estimate AMM liquidity
        amm_liquidity = uniswap_data.get_pool_liquidity_depth(
            pool_id, 
            max_price_impact_pct=risk_params['max_price_impact_pct']
        )
        print(f"\nAMM liquidity at {risk_params['max_price_impact_pct']}% impact: ${amm_liquidity['max_trade_size_usd']:,.0f}")
    else:
        print("No pool data available")
        amm_liquidity = {'max_trade_size_usd': 0}
        
except Exception as e:
    print(f"Error fetching Uniswap data: {e}")
    amm_liquidity = {'max_trade_size_usd': 0}

## 5. Volatility Analysis

In [ ]:
# Calculate volatility metrics
vol_30d = pricing.realized_vol(klines_1h, window=30*24, freq_per_day=24)
vol_7d = pricing.realized_vol(klines_1h, window=7*24, freq_per_day=24)
vol_1d = pricing.realized_vol(klines_1h, window=24, freq_per_day=24)

print(f"Realized Volatility:")
print(f"  - 30-day: {vol_30d*100:.1f}%")
print(f"  - 7-day: {vol_7d*100:.1f}%")
print(f"  - 1-day: {vol_1d*100:.1f}%")

# GARCH-style forecast
garch_vol = pricing.garch_vol(klines_1h)
print(f"\nVolatility forecast:")
print(f"  - Current (EWMA): {garch_vol['current_vol']*100:.1f}%")
print(f"  - Forecast: {garch_vol['forecast_vol']*100:.1f}%")

In [ ]:
# Plot volatility history
fig, axes = plotting.plot_volatility_history(klines_1h, window=24*7)
plt.show()

## 6. Combined Liquidity Analysis

In [ ]:
# Combine CEX and AMM liquidity
cex_depth = depths[risk_params['max_price_impact_pct']]
amm_depth = amm_liquidity['max_trade_size_usd']

combined_liquidity = pricing.combine_liquidity_sources(
    cex_depth_usd=cex_depth,
    amm_depth_usd=amm_depth,
    amm_haircut=0.7  # 30% haircut for AMM liquidity
)

print(f"Combined Liquidity Analysis:")
print(f"  - CEX liquidity: ${combined_liquidity['cex_liquidity_usd']:,.0f}")
print(f"  - AMM liquidity (raw): ${combined_liquidity['amm_liquidity_usd']:,.0f}")
print(f"  - AMM liquidity (adjusted): ${combined_liquidity['amm_adjusted_usd']:,.0f}")
print(f"  - Total liquidity: ${combined_liquidity['total_liquidity_usd']:,.0f}")
print(f"  - CEX share: {combined_liquidity['cex_share']*100:.1f}%")
print(f"  - AMM share: {combined_liquidity['amm_share']*100:.1f}%")

## 7. Summary Statistics

In [ ]:
# Create summary DataFrame
summary_data = {
    'Market': market['name'],
    'Symbol': symbol,
    'Current Price': klines_1h['close'].iloc[-1],
    '24h Volume (USD)': float(ticker_24h['quoteVolume']),
    'Spread (bps)': spread_metrics['spread_bps'],
    'CEX Liquidity @ 1%': cex_depth,
    'AMM TVL': float(pool_data.get('totalValueLockedUSD', 0)) if 'pool_data' in locals() else 0,
    'Total Liquidity': combined_liquidity['total_liquidity_usd'],
    '30d Volatility': vol_30d * 100,
    '7d Volatility': vol_7d * 100
}

summary_df = pd.DataFrame([summary_data])
summary_df

## 8. Multi-Market Comparison

In [ ]:
# Analyze multiple markets
markets_analysis = {}

for market in markets_config['markets'][:3]:  # Analyze first 3 markets
    try:
        print(f"\nAnalyzing {market['name']}...")
        symbol = market['binance_symbol_perp']
        
        # Fetch data
        orderbook = binance.get_orderbook(symbol, limit=100, use_futures=True)
        klines = binance.get_klines(symbol, interval='1h', lookback_days=7, use_futures=True)
        ticker = binance.get_ticker_24h(symbol, use_futures=True)
        
        # Calculate metrics
        liq_curve = pricing.compute_orderbook_liquidity_curve(orderbook)
        cex_liq = pricing.depth_at_impact(liq_curve, 1.0)
        vol = pricing.realized_vol(klines) * 100
        
        markets_analysis[market['name']] = {
            'cex_liquidity_usd': cex_liq,
            'volume_24h_usd': float(ticker['quoteVolume']),
            'volatility_7d': vol,
            'total_liquidity_usd': cex_liq  # Simplified for now
        }
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

In [ ]:
# Compare liquidity across markets
if len(markets_analysis) > 1:
    fig, axes = plotting.plot_liquidity_comparison(markets_analysis)
    plt.show()

## Conclusions

This exploration provides insights into:

1. **Liquidity Depth**: How much can be traded before significant price impact
2. **Volatility Patterns**: Historical and forecast volatility for risk assessment
3. **Market Microstructure**: Spreads, order book shape, and liquidity distribution
4. **CEX vs AMM**: Relative importance of different liquidity sources

Next steps:
- Use these metrics to calculate risk scores and OI limits (notebook 01)
- Run stress scenarios based on current market conditions (notebook 02)